#### This script will remove not used functions from sample projects

In [1]:
import sys
sys.path.insert(0,'..')

import os
import json
from controller.transfer import project_transfer_manager as pm
import umap
import numpy as np
from zipfile import ZipFile

path = "../sample_projects/"

# List files ending with .zip
files = [f for f in os.listdir(path) if f.endswith(".zip")]

/Users/ibiscp/GitHub/logspace/alfred/modules/refinery-gateway/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# files = ['clickbait.zip']

In [ ]:
# Generate reduced embeddings
trans = umap.UMAP(n_neighbors=10,
        min_dist=0.1,
        n_components=2,
        metric="euclidean",
        random_state=42)

In [ ]:
for file in files:
    # Extract zip file
    data = pm.extract_first_zip_data(f"{path}{file}")

    # for key in data.keys():
    #     print(key)
    
    # Remove labeling tasks
    # Labeling Tasks
    lt_ids = [lt['id'] for lt in data['labeling_tasks_data'] if lt['task_type'] != 'MULTICLASS_CLASSIFICATION']

    # Labeling task labels
    ltl_ids = [ltl['id'] for ltl in data['labeling_task_labels_data'] if ltl['labeling_task_id'] in lt_ids]

    # Information Source
    is_ids = [is_['id'] for is_ in data['information_sources_data'] if is_['type'] != 'ACTIVE_LEARNING']

    # Information Source Payload
    isp_ids = [isp['id'] for isp in data['information_source_payloads_data'] if isp['source_id'] in is_ids]

    # Information Source Statistics
    iss_ids = [iss['id'] for iss in data['information_source_statistics_data'] if iss['source_id'] in is_ids or iss['labeling_task_label_id'] in ltl_ids]

    # Record label associations
    rla_ids = [rla['id'] for rla in data['record_label_associations_data'] if rla['source_id'] in is_ids or rla['labeling_task_label_id'] in ltl_ids]

    # Record label association token
    rlat = [rlat for rlat in data['record_label_association_tokens_data'] if rlat['record_label_association_id'] not in rla_ids]
    

    # Remove everything
    data['record_label_association_tokens_data'] = rlat
    data['record_label_associations_data'] = [rla for rla in data['record_label_associations_data'] if rla['id'] not in rla_ids]
    data['information_source_statistics_data'] = [iss for iss in data['information_source_statistics_data'] if iss['id'] not in iss_ids]
    data['information_source_payloads_data'] = [isp for isp in data['information_source_payloads_data'] if isp['id'] not in isp_ids]
    data['information_sources_data'] = [is_ for is_ in data['information_sources_data'] if is_['id'] not in is_ids]
    data['labeling_task_labels_data'] = [ltl for ltl in data['labeling_task_labels_data'] if ltl['id'] not in ltl_ids]
    data['labeling_tasks_data'] = [lt for lt in data['labeling_tasks_data'] if lt['id'] not in lt_ids]

    if data["embedding_tensors_data"]:
        embedding_vector = np.array([e['data'] for e in data["embedding_tensors_data"]])
        data_reduced = trans.fit_transform(embedding_vector)

        for i, e in enumerate(data["embedding_tensors_data"]):
            e['data_reduced'] = data_reduced[i].tolist()

    # Save file
    with ZipFile(f"{path}changed/{file}", 'w') as myzip:
        with myzip.open("config.json", "w") as c:
            c.write(json.dumps(data).encode("utf-8"))

    # break